# Topic Modeling in Python
In diesem Notebook wird in Kürze das Topic Modeling anhand der Sturm-Briefedition vorgeführt.

In einem ersten Schritt extrahieren wir den Text ohne Annotationen aus den XML-Dokumenten, in einem zweiten Schritt führen wir das Topic Modeling durch.

In [ ]:
%pip install -U lxml

In [ ]:
# import der relevanten Bibliotheken
from lxml import etree as et
import glob
import re

texts = []

for letter in glob.glob("docs/*.xml"):
    tree = et.parse(letter)
    root = tree.getroot()

    # den Haupttext greifen
    paragraphs = root.xpath("./tei:text/tei:body/tei:div[@type='content']/tei:p", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})

    for paragraph in paragraphs:
        # den Text von Tags und Kommentaren bereinigen
        et.strip_elements(paragraph, "{http://www.tei-c.org/ns/1.0}note", with_tail=False)
        paragraph_text = et.tostring(paragraph, method="text", encoding="utf8", with_tail=False).decode("utf8")
        texts.append(paragraph_text)

print(texts)

Die Texte möchten wir nun noch vorverarbeiten. Dazu gehört einerseits das *tokenisieren* also auflösen in Worteinheiten, das Entfernen von Stoppwörtern und die Lemmatisierung von Wörtern, also die Reduktion auf eine Grundform (z.B. Infinitiv).

Für das Preprocessing empfiehlt sich bei grösseren Projekten eine NLP-Bibliothek wie [spacy](https://spacy.io/). Spacy ist eine NLP-Bibliothek, die eine komplette Pipeline, u.a. auch Named-Entity- und POS-Tagging unterstützt.

In [ ]:
%pip install -U spacy

# und das deutsche Modell
!python -m spacy download de_core_news_sm

In [ ]:
# Für Spacy müssen wir ein bestimmtes Modell herunterladen, mit dem wir die Texte vorbereiten wollen.
# Wir nehmen für die Sturm-Edition das kleine deutsche Modell, das ist auch schnell heruntergeladen
# Wir schalten aber die Module ab, die wir nicht benötigen, damit die Texte schneller bearbeitet werden können.
import spacy

nlp = spacy.load("de_core_news_sm", disable=["parser", "ner"])

In [ ]:
def preprocess(text):
    doc = nlp(text.lower())
    # wir geben das Lemma zurück falls es kein Stopwort ist und alphanumerisch ist
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

# Listenkomprehension ersetzt hier eine for-Schleife, tut aber dasselbe
prepped_texts = [preprocess(text) for text in texts]

print(prepped_texts)

Für das Topic Modeling verwenden wir hier [Tomotopy](https://bab2min.github.io/tomotopy/v0.13.0/en/), ein Package, das in Python sehr einfach anwendbar ist.

In [ ]:
%pip install -U tomotopy

In [ ]:
import tomotopy as tp

# Initialisiere das Modell, k gibt die Zahl der gewünschten Topics an
mdl = tp.LDAModel(k=5)

# Lade alle Texte ins Modell
for text in prepped_texts:
    mdl.add_doc(text)

# Trainiere das Modell, alle 100 Trainings-Iterationen wird der Stand ausgegeben
for i in range(0, 1000, 100):
    mdl.train(100)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

# Zeige, welche Topics mit welchen wichtigen Wörtern erkannt wurden
for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))

mdl.summary()

Im Viewer können wir unser Modell inspizieren. Mit der geringen Menge an Dokumenten in unserem Fall ist es nicht überraschend, dass die Topics eher ein durcheinander sind!

In [ ]:
tp.viewer.open_viewer(mdl, port=9999)